# Modelo Predicción de Ocupación
### Ejecución de modelo entrenado

## Contenido
* Parámetros generales e Importacion de biblioticas
* Carga de los datos
* Parámetros Generales
* Preprocesamiento
 * Unificación 
* Modelo
 * Procesamiento
 * Tidy Table
 * Separación Xvar y Yvar  (número variable de periodo)
 * Separacion Train y Test
 * Configuración de los Modelos
 * Funciones 
  * F. para evaluar varios modelos
  * F. para categorizar en semáforo
  * F. Evaluación de resultados Exactitud
* Generacion de Pronosticos a futuro
 * Procesamiento entrada para pronóstico
 * Ejecución del modelos y archivo de entrega

## Parámetros generales e Importacion de biblioticas

In [1]:
%load_ext autoreload
%autoreload 2
np.set_printoptions(suppress=True)

NameError: name 'np' is not defined

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import shap
from datetime import datetime
#from datetime import timedelta
from dateutil.relativedelta import relativedelta
#import datetime
from dateutil.relativedelta import relativedelta
from imblearn.over_sampling import SMOTE
import xgboost as xgb
from xgboost import plot_importance
from lightgbm import LGBMClassifier
from sklearn.metrics import confusion_matrix # es una manera para resumir los datos 
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_curve #roc_curve
from sklearn.metrics import roc_auc_score #auc
from joblib import dump,load

## Carga de los datos

In [3]:
#df_Nodosokla= pd.read_csv('DatosOrigen/NODOS_INFO_OOKLA.csv',sep=';', header = 0)
data_ocupacionNodo = pd.read_csv('DatosOrigen/OCUPACION_NODO.csv', sep=';')
data_NodoINFI = pd.read_csv('DatosOrigen/NODO_INFI.csv', sep=';')
data_SEG = pd.read_csv('DatosOrigen/segmentacion.csv', sep=';') 

### Base OOKLA

In [4]:
"""
df_Nodosokla['DIA']=1 # le agrego la columna dia
df_Nodosokla['MES'] = [f'0{x}' if x <10 else str(x) for x in df_Nodosokla['MES']]
df_Nodosokla['DIA'] = [f'0{x}' if x <10 else str(x) for x in df_Nodosokla['DIA']]
df_Nodosokla['FECHA'] = df_Nodosokla['ANIO'].astype('str') + '-' + df_Nodosokla['MES'] + '-' + df_Nodosokla['DIA']
# eliminamos columnas cobrantes
df_Nodosokla=df_Nodosokla.drop(['DIA','MES','ANIO'], axis=1)
# ordenamos
df_Nodosokla = df_Nodosokla[['NODO','FECHA', 'OPERADOR','CANTIDAD','DOWNLOAD_KBPS','UPLOAD_KBPS','LATENCY','FLAG_LAT']] 
# # renombramos columnas
df_Nodosokla.rename(columns={'NODO':'COD_NODO'}, inplace=True)
df_Nodosokla_Claro=df_Nodosokla[df_Nodosokla["OPERADOR"]=="CLARO"]
del(df_Nodosokla_Claro['OPERADOR'])"""

'\ndf_Nodosokla[\'DIA\']=1 # le agrego la columna dia\ndf_Nodosokla[\'MES\'] = [f\'0{x}\' if x <10 else str(x) for x in df_Nodosokla[\'MES\']]\ndf_Nodosokla[\'DIA\'] = [f\'0{x}\' if x <10 else str(x) for x in df_Nodosokla[\'DIA\']]\ndf_Nodosokla[\'FECHA\'] = df_Nodosokla[\'ANIO\'].astype(\'str\') + \'-\' + df_Nodosokla[\'MES\'] + \'-\' + df_Nodosokla[\'DIA\']\n# eliminamos columnas cobrantes\ndf_Nodosokla=df_Nodosokla.drop([\'DIA\',\'MES\',\'ANIO\'], axis=1)\n# ordenamos\ndf_Nodosokla = df_Nodosokla[[\'NODO\',\'FECHA\', \'OPERADOR\',\'CANTIDAD\',\'DOWNLOAD_KBPS\',\'UPLOAD_KBPS\',\'LATENCY\',\'FLAG_LAT\']] \n# # renombramos columnas\ndf_Nodosokla.rename(columns={\'NODO\':\'COD_NODO\'}, inplace=True)\ndf_Nodosokla_Claro=df_Nodosokla[df_Nodosokla["OPERADOR"]=="CLARO"]\ndel(df_Nodosokla_Claro[\'OPERADOR\'])'

### Base data_NodoINF

In [5]:
data_NodoINFI['DIA']=1 # le agrego la columna dia
data_NodoINFI['MES'] = [f'0{x}' if x <10 else str(x) for x in data_NodoINFI['MES']]
data_NodoINFI['DIA'] = [f'0{x}' if x <10 else str(x) for x in data_NodoINFI['DIA']]
data_NodoINFI['FECHA'] = (2000+data_NodoINFI['ANIO']).astype('str') + '-' + data_NodoINFI['MES'] + '-' + data_NodoINFI['DIA']
# eliminamos columnas sobrantes
data_NodoINFI=data_NodoINFI.drop(['DIA','MES','ANIO'], axis=1)
# # renombramos columnas
data_NodoINFI.rename(columns={'NODO':'COD_NODO'}, inplace=True)
### completar Nullos con 0
data_NodoINFI=data_NodoINFI.fillna(0)

### Base data_NodoINF data_SEG

In [6]:
data_SEG.drop(['NUEVA ESTRUCTURA'], axis=1,inplace=True)
data_SEG['SEGMENTADO']=1

## Preprocesamiento
### Unificacion

In [7]:
data_ocupacion=pd.merge(data_ocupacionNodo, data_NodoINFI, on = ('COD_NODO','FECHA'), how='left')
#data_ocupacion=pd.merge(data_ocupacion, df_Nodosokla_Claro, on = ('COD_NODO','FECHA'), how='left')
data_ocupacion=pd.merge(data_ocupacion, data_SEG, on = ('COD_NODO','FECHA'), how='left')

In [8]:
# crear factor beta y delta

In [9]:
data_ocupacion.head(2)

,COD_NODO,FECHA,OCUPACION_DW,OCUPACION_UP,CANT_USER,PROM_VEL,MEDIANA_VEL,VEL_ME_8,VEL_10_25,VEL_30,...,VEL_MAX,VEL_MIN,RENTA,PROM_RENTA,CONVERGENTE,SERV_INT,SERV_TV,SERV_VOZ,CANT_CALL,SEGMENTADO
0,PKE,2020-06-01,0.355112,0.124435,90.0,17.0,10.0,32.0,31.0,16.0,...,50.0,3.0,7925605.0,88062.0,22.0,90.0,83.0,83.0,4.0,NaN
1,2AU3,2020-06-01,0.208285,0.242637,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### Parámetros generales

In [10]:
start = datetime.now()

In [11]:
fechaMaxima=datetime.strptime(data_ocupacion['FECHA'].max(),'%Y-%m-%d')
periodos_y=7
periodos_x=7
variable_prediccion='OCUPACION_DW'

In [12]:
fechaLimite=fechaMaxima + relativedelta(months=-1*periodos_y)
fechaInicial=fechaLimite + relativedelta(months=-1*periodos_x)

### Selección Fechas

In [13]:
## Eliminar informaición anterior Junion 2022
data_ocupacion['FECHA']=data_ocupacion['FECHA'].astype('datetime64[ns]')
### Elininar datos últimos mes
data_ocupacion=data_ocupacion[data_ocupacion['FECHA']>=fechaInicial]
### Eliminar Fecha Final  (mes incompleto)
##  data_ocupacion=data_ocupacion[data_ocupacion['FECHA']<fechaMaxima]

##  Funciones

In [14]:
#Funcion para Calcular el número de periodos hacia atrás
def diff_month(d1, d2):
    return (d1.year - d2.dt.year) * 12 + d1.month - d2.dt.month

In [15]:
def semaforo(df, col):
    df = df.copy()
    df.loc[(df[col]>=0.58) & (df[col]<=0.75), 'ESTATUS'] = 'AMARILLO'
    df.loc[df[col]>0.75, 'ESTATUS'] = 'ROJO'
    df.loc[df[col]<0.60, 'ESTATUS'] = 'VERDE'
    return(df)

In [16]:
def semaforoVal(val):
    #df = df.copy()
    respuesta='V'
    if (val>=0.58) & (val<=0.75):
        respuesta='A'
    elif val>0.75:
        respuesta='R'
    return(respuesta)

In [17]:
def semaforoNum2Val(val):   
    respuesta='V'
    if val==1:
        respuesta='A'
    elif val==2:
        respuesta='R'
    return(respuesta)

In [18]:
def semaforoVal2Num(val):   
    respuesta=0
    if val=='A':
        respuesta=1
    elif val=='R':
        respuesta=2
    return(respuesta)

In [19]:
def semaforoRojo(val):    
    if val>0.75:
        respuesta=1
    else :
        respuesta=0
    return(respuesta)

In [20]:
def semaforoAmarillo(val):    
    if (val>=0.58) & (val<=0.75):
        respuesta=1
    else :
        respuesta=0
    return(respuesta)

In [21]:
def semaforoRojoAmarillo(val):    
    if val>=0.58:
        respuesta=1
    else :
        respuesta=0
    return(respuesta)

In [22]:
def semaforoNum(val):
    #df = df.copy()
    respuesta=0
    if (val>=0.58) & (val<=0.75):
        respuesta=1
    elif val>0.75:
        respuesta=2
    return(respuesta)

In [23]:
def semaforoNum2Rojo(val):    
    respuesta=0
    if val==2:
        respuesta=1    
    return(respuesta)

In [24]:
def semaforoNum2Amarillo(val):    
    respuesta=0
    if val==1:
        respuesta=1    
    return(respuesta)

In [25]:
def semaforoNum2RojoAmarillo(val):    
    respuesta=0
    if val>=1:
        respuesta=1    
    return(respuesta)

In [26]:
def medidasEvaluacion(matrizConfucion):
    exactitud=np.trace(matrizConfucion)/matrizConfucion.sum()
    print ('exactitud:', exactitud)
    sensibilidad=[0]*3
    especificidad=[0]*3
    for i in range(0,len(matrizConfucion)):
        sensibilidad[i]=matrizConfucion[i,i]/matrizConfucion.sum(axis=1)[i]
        especificidad[i]=np.delete(np.delete(matrizConfucion,i, axis=0),i,axis=1).sum()/np.delete(matrizConfucion,i, axis=0).sum()
        print ('sensibilidad ',i,": ",sensibilidad[i])
        print ('especificidad ',i,": ",especificidad[i])

In [27]:
def sampling_strategy(X,y,n_samples,t='majority'):
    target_classes=''
    if t== 'majority':
        target_classes= y.value_counts() >n_samples
    elif t== 'minority':
        target_classes = y.value_counts() < n_samples
    tc=target_classes[target_classes ==True].index
    #target_classes_all=y.value_counts().index
    sampling_strategy={}
    for target in tc:
        sampling_strategy[target]= n_samples
    return sampling_strategy

In [28]:
def df2Feature_importanceClass(x_train_b,mod,title):
    #shap_test = x_train_b.sample(1000)
    shap_test = x_train_b
    shap_values = shap.TreeExplainer(mod).shap_values(shap_test)
    plt.figure()
    plt.title(title)
    shap.summary_plot(shap_values, shap_test,max_display=30)
    plt.subplots_adjust(left=0.35, right=0.9, top=0.9, bottom=0.3)    
    plt.show();

## Modelo
### Procesamiento

In [29]:
data_ocupacion['mesAnt']=(periodos_x+periodos_y)-diff_month(fechaMaxima,data_ocupacion['FECHA'])
### Ordenar por fecha
data_ocupacion=data_ocupacion.sort_values(["FECHA","COD_NODO"], ascending = (True, True))

### Seleccion Nodos para el Modelo

In [30]:
### Solo Nodos con información completa
seleccion=data_ocupacion[['COD_NODO',variable_prediccion]].groupby('COD_NODO').count()
seleccion[seleccion[variable_prediccion]>=periodos_y+periodos_x+1]
seleccion.rename(columns={variable_prediccion:'noRegVarPre'}, inplace=True)
data_ocupacion=data_ocupacion.merge(seleccion, on='COD_NODO', how='left')
data_ocupacion=data_ocupacion[data_ocupacion['noRegVarPre']>=periodos_y+periodos_x]
data_ocupacion.drop(columns='noRegVarPre', inplace=True)
### Nodos No Segmentados en el periodo de tiempo considerado
data_ocupacion=data_ocupacion[data_ocupacion['SEGMENTADO']!=1]
data_ocupacion.drop(columns='SEGMENTADO', inplace=True)
### Eliminar datos con nullos
data_ocupacion=data_ocupacion.dropna()

###  Selección Columnas

In [31]:
selCols=['COD_NODO','FECHA','OCUPACION_DW','OCUPACION_UP','SERV_INT','mesAnt']
data_ocupacionC=data_ocupacion.copy()
data_ocupacion=data_ocupacion[selCols]

### Tidy Tables

#### Regresión

In [32]:
cols=data_ocupacion.columns
cols=cols[2:-1]

In [33]:
cols_x=[]
for i, col in enumerate(cols):
    tablaAnalisis = pd.pivot_table(data_ocupacion.reset_index(), index='COD_NODO', columns='mesAnt', values=col)
    tablaAnalisis.rename(columns={x: (col+"-"+str(x)) for x in tablaAnalisis.columns}, inplace=True)
    #Identificacion Columnas X y Y    
    cols_x=cols_x+[(col+"-"+str(x)) for x in range(0,periodos_x)]    
    if i==0:
        tablaAnalisisTotal = tablaAnalisis
    else:
        tablaAnalisisTotal = tablaAnalisisTotal.merge(tablaAnalisis, how='outer', on='COD_NODO')

In [34]:
cols_x=sorted(list(set(cols_x)& set(tablaAnalisisTotal.columns.tolist())))

In [35]:
cols_y=[(variable_prediccion+"-"+str(x)) for x in range(periodos_x,periodos_x+periodos_y-1)]

In [36]:
yColClass=[s + '_sem2' for s in cols_y]

In [37]:
tablaAnalisisTotal=tablaAnalisisTotal.dropna()

In [38]:
data_ocupacionC.head()

,COD_NODO,FECHA,OCUPACION_DW,OCUPACION_UP,CANT_USER,PROM_VEL,MEDIANA_VEL,VEL_ME_8,VEL_10_25,VEL_30,...,VEL_MAX,VEL_MIN,RENTA,PROM_RENTA,CONVERGENTE,SERV_INT,SERV_TV,SERV_VOZ,CANT_CALL,mesAnt
0,01J,2020-10-01,0.290939,0.212174,26.0,40.0,30.0,2.0,3.0,9.0,...,100.0,5.0,2694135.0,103621.0,6.0,26.0,21.0,18.0,1.0,0
1,01S,2020-10-01,0.358060,0.219646,146.0,34.0,30.0,46.0,6.0,30.0,...,150.0,5.0,13139893.0,89999.0,77.0,146.0,109.0,120.0,21.0,0
3,02J,2020-10-01,0.290939,0.212174,9.0,44.0,45.0,0.0,2.0,0.0,...,100.0,10.0,951287.0,105699.0,4.0,9.0,6.0,8.0,2.0,0
4,02S,2020-10-01,0.358060,0.229191,116.0,28.0,30.0,36.0,12.0,29.0,...,100.0,5.0,10170582.0,87677.0,37.0,116.0,83.0,97.0,11.0,0
5,03J,2020-10-01,0.290939,0.212174,28.0,35.0,35.0,1.0,6.0,7.0,...,75.0,5.0,3101336.0,110762.0,12.0,28.0,26.0,24.0,4.0,0


#### Clasificación

In [39]:
colsC=data_ocupacionC.columns
colsC=colsC[2:-1]

In [40]:
cols_xC=[]
for i, col in enumerate(colsC):
    tablaAnalisis = pd.pivot_table(data_ocupacionC.reset_index(), index='COD_NODO', columns='mesAnt', values=col)
    tablaAnalisis.rename(columns={x: (col+"-"+str(x)) for x in tablaAnalisis.columns}, inplace=True)
    #Identificacion Columnas X y Y    
    cols_xC=cols_xC+[(col+"-"+str(x)) for x in range(0,periodos_x)]    
    if i==0:
        tablaAnalisisTotalC = tablaAnalisis
    else:
        tablaAnalisisTotalC = tablaAnalisisTotalC.merge(tablaAnalisis, how='outer', on='COD_NODO')
tablaAnalisisTotalC=tablaAnalisisTotalC.dropna()

In [41]:
cols_xC=sorted(list(set(cols_xC)& set(tablaAnalisisTotalC.columns.tolist())))

## Carga de los modelos

In [42]:
ruta_objeto = f"modeloR.pkl"
modeloR = load(ruta_objeto)

In [43]:
ruta_objeto = f"modeloC.pkl"
modeloC = load(ruta_objeto)

### Selección datos Base

In [44]:
data_ocupacionNp=data_ocupacion.copy()
data_ocupacionNpC=data_ocupacionC.copy()

In [45]:
periodos_x=6
fechaMaxima=data_ocupacionNp['FECHA'].max()
fechaInicialNp=fechaMaxima + relativedelta(months=-1*periodos_x)

In [46]:
fechaMaxima

Timestamp('2021-12-01 00:00:00')

In [47]:
#meses a predecir
date_list = [(fechaMaxima + relativedelta(months=+x+1)).strftime('%b-%y') for x in range(6)]

In [48]:
date_list

['Jan-22', 'Feb-22', 'Mar-22', 'Apr-22', 'May-22', 'Jun-22']

In [49]:
data_ocupacionNp=data_ocupacionNp[data_ocupacionNp['FECHA']>=fechaInicialNp]
data_ocupacionNpC=data_ocupacionNpC[data_ocupacionNpC['FECHA']>=fechaInicialNp]
data_ocupacionNp['mesAnt']=(periodos_x)-diff_month(fechaMaxima,data_ocupacionNp['FECHA'])
data_ocupacionNpC['mesAnt']=(periodos_x)-diff_month(fechaMaxima,data_ocupacionNpC['FECHA'])

In [50]:
colsNp=data_ocupacionNp.columns
colsNp=colsNp[2:-1]
colsNpC=data_ocupacionNpC.columns
colsNpC=colsNpC[2:-1]

In [51]:
#Tabla para regresión
cols_xNp=[]
for i, col in enumerate(colsNp):
    tablaAnalisisNp = pd.pivot_table(data_ocupacionNp.reset_index(), index='COD_NODO', columns='mesAnt', values=col)
    tablaAnalisisNp.rename(columns={x: (col+"-"+str(x)) for x in tablaAnalisisNp.columns}, inplace=True)    
    cols_xNp=cols_xNp+[(col+"-"+str(x)) for x in range(0,periodos_x+1)]    
    if i==0:
        tablaAnalisisTotalNp = tablaAnalisisNp
    else:
        tablaAnalisisTotalNp = tablaAnalisisTotalNp.merge(tablaAnalisisNp, how='outer', on='COD_NODO')

In [52]:
#Tabla para clasificación
cols_xNpC=[]
for i, col in enumerate(colsNpC):
    tablaAnalisisNpC = pd.pivot_table(data_ocupacionNpC.reset_index(), index='COD_NODO', columns='mesAnt', values=col)
    tablaAnalisisNpC.rename(columns={x: (col+"-"+str(x)) for x in tablaAnalisisNpC.columns}, inplace=True)    
    cols_xNpC=cols_xNpC+[(col+"-"+str(x)) for x in range(0,periodos_x+1)]    
    if i==0:
        tablaAnalisisTotalNpC = tablaAnalisisNpC
    else:
        tablaAnalisisTotalNpC = tablaAnalisisTotalNpC.merge(tablaAnalisisNpC, how='outer', on='COD_NODO')

In [53]:
tablaAnalisisTotalNp=tablaAnalisisTotalNp.dropna()
tablaAnalisisTotalNpC=tablaAnalisisTotalNpC.dropna()

In [54]:
tablaResultados=tablaAnalisisTotalNp.copy()

In [55]:
#for predVar in modelos:
for predVar in cols_y:
    print(predVar)
    tablaResultados[predVar+'_pR']=modeloR[predVar].predict(tablaAnalisisTotalNp[cols_x])
    tablaResultados[predVar+'_pR_Sem']=tablaResultados[predVar+'_pR'].apply(semaforoVal)  

OCUPACION_DW-7
OCUPACION_DW-8
OCUPACION_DW-9
OCUPACION_DW-10
OCUPACION_DW-11
OCUPACION_DW-12


In [56]:
#for predVar in modelosClass:
for predVar in yColClass: 
    print(predVar)
    tablaResultados[predVar+'_pC']=modeloC[predVar].predict(tablaAnalisisTotalNpC[cols_xC])
    tablaResultados[predVar+'_pC_probR']=modeloC[predVar].predict_proba(tablaAnalisisTotalNpC[cols_xC])[:,2]
    tablaResultados[predVar+'_pC_probA']=modeloC[predVar].predict_proba(tablaAnalisisTotalNpC[cols_xC])[:,1]
    tablaResultados[predVar+'_pC']=tablaResultados[predVar+'_pC'].apply(semaforoNum2Val)    

OCUPACION_DW-7_sem2
OCUPACION_DW-8_sem2
OCUPACION_DW-9_sem2
OCUPACION_DW-10_sem2
OCUPACION_DW-11_sem2
OCUPACION_DW-12_sem2


In [57]:
import pandas as pd
pd.set_option('display.max_columns', None)
tablaResultados.head(10)

mesAnt,OCUPACION_DW-0,OCUPACION_DW-1,OCUPACION_DW-2,OCUPACION_DW-3,OCUPACION_DW-4,OCUPACION_DW-5,OCUPACION_DW-6,OCUPACION_UP-0,OCUPACION_UP-1,OCUPACION_UP-2,OCUPACION_UP-3,OCUPACION_UP-4,OCUPACION_UP-5,OCUPACION_UP-6,SERV_INT-0,SERV_INT-1,SERV_INT-2,SERV_INT-3,SERV_INT-4,SERV_INT-5,SERV_INT-6,OCUPACION_DW-7_pR,OCUPACION_DW-7_pR_Sem,OCUPACION_DW-8_pR,OCUPACION_DW-8_pR_Sem,OCUPACION_DW-9_pR,OCUPACION_DW-9_pR_Sem,OCUPACION_DW-10_pR,OCUPACION_DW-10_pR_Sem,OCUPACION_DW-11_pR,OCUPACION_DW-11_pR_Sem,OCUPACION_DW-12_pR,OCUPACION_DW-12_pR_Sem,OCUPACION_DW-7_sem2_pC,OCUPACION_DW-7_sem2_pC_probR,OCUPACION_DW-7_sem2_pC_probA,OCUPACION_DW-8_sem2_pC,OCUPACION_DW-8_sem2_pC_probR,OCUPACION_DW-8_sem2_pC_probA,OCUPACION_DW-9_sem2_pC,OCUPACION_DW-9_sem2_pC_probR,OCUPACION_DW-9_sem2_pC_probA,OCUPACION_DW-10_sem2_pC,OCUPACION_DW-10_sem2_pC_probR,OCUPACION_DW-10_sem2_pC_probA,OCUPACION_DW-11_sem2_pC,OCUPACION_DW-11_sem2_pC_probR,OCUPACION_DW-11_sem2_pC_probA,OCUPACION_DW-12_sem2_pC,OCUPACION_DW-12_sem2_pC_probR,OCUPACION_DW-12_sem2_pC_probA
COD_NODO,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
01J,0.337471,0.350570,0.314207,0.332251,0.319572,0.326798,0.297562,0.220097,0.233507,0.257879,0.232684,0.179844,0.246624,0.185259,25.0,24.0,23.0,24.0,24.0,24.0,25.0,0.326445,V,0.332595,V,0.342219,V,0.317749,V,0.333805,V,0.340452,V,V,0.001015,0.001672,V,0.009629,0.017980,V,0.020471,0.016030,V,0.019062,0.037177,V,0.008415,0.026106,V,0.008581,0.046267
01S,0.348979,0.432986,0.404191,0.379371,0.394562,0.422378,0.427491,0.306406,0.269256,0.287102,0.310938,0.346350,0.312825,0.270177,170.0,177.0,175.0,186.0,184.0,177.0,187.0,0.452395,V,0.433711,V,0.428239,V,0.421707,V,0.451648,V,0.468728,V,V,0.001112,0.001561,V,0.019364,0.016871,V,0.010402,0.017130,V,0.011088,0.042812,V,0.018257,0.022082,V,0.011535,0.024536
02J,0.337471,0.350570,0.314207,0.332251,0.319572,0.326798,0.297562,0.220097,0.233507,0.257879,0.232684,0.179844,0.246624,0.185259,14.0,15.0,13.0,12.0,11.0,12.0,12.0,0.325362,V,0.334995,V,0.342638,V,0.332294,V,0.339170,V,0.337937,V,V,0.001185,0.002022,V,0.018874,0.105148,V,0.019484,0.032147,V,0.025898,0.093686,V,0.034324,0.103864,V,0.024198,0.280474
02S,0.348979,0.432986,0.404191,0.379371,0.394562,0.422378,0.427491,0.243914,0.227977,0.228819,0.247392,0.231056,0.270342,0.225271,144.0,148.0,162.0,162.0,147.0,144.0,154.0,0.458008,V,0.454372,V,0.436034,V,0.436655,V,0.457271,V,0.460827,V,V,0.000902,0.001515,V,0.018122,0.016214,V,0.007959,0.014188,V,0.009991,0.036683,V,0.009672,0.038803,V,0.010654,0.034737
03J,0.337471,0.350570,0.314207,0.332251,0.319572,0.326798,0.297562,0.220097,0.233507,0.257879,0.232684,0.179844,0.246624,0.185259,32.0,30.0,29.0,29.0,28.0,26.0,27.0,0.326517,V,0.332146,V,0.347058,V,0.320673,V,0.335939,V,0.337931,V,V,0.000884,0.001427,V,0.005090,0.015696,V,0.013980,0.006277,V,0.013553,0.035627,V,0.006401,0.025269,V,0.004983,0.043769
04J,0.337471,0.350570,0.314207,0.332251,0.319572,0.326798,0.297562,0.220097,0.233507,0.257879,0.232684,0.179844,0.246624,0.185259,21.0,20.0,20.0,20.0,18.0,18.0,19.0,0.326650,V,0.333017,V,0.340039,V,0.325808,V,0.338218,V,0.341460,V,V,0.001032,0.001323,V,0.020206,0.036398,V,0.017286,0.034538,V,0.065541,0.156788,V,0.059087,0.258567,V,0.047869,0.455737
05J,0.318871,0.289796,0.289193,0.260464,0.288334,0.278748,0.239683,0.257956,0.237106,0.280441,0.264702,0.325102,0.305621,0.273278,60.0,61.0,58.0,57.0,54.0,50.0,54.0,0.264884,V,0.271320,V,0.299118,V,0.301514,V,0.301772,V,0.293222,V,V,0.000941,0.001567,V,0.004648,0.018520,V,0.025769,0.042401,V,0.013914,0.023953,V,0.003617,0.022076,V,0.008564,0.026068
06J,0.318871,0.289796,0.289193,0.260464,0.288334,0.278748,0.239683,0.257956,0.237106,0.280441,0.264702,0.325102,0.305621,0.273278,40.0,39.0,37.0,37.0,37.0,39.0,39.0,0.261747,V,0.258449,V,0.298773,V,0.300777,V,0.304385,V,0.295104,V,V,0.000946,0.001578,V,0.004728,0.014483,V,0.028383,0.024177,V,0.015195,0.016514,V,0.009034,0.024172,V,0.022949,0.036752
07J,0.318871,0.289796,0.289193,0.260464,0.288334,0.278748,0.239683,0.257956,0.23710

####  Ranking de Nodos

In [58]:
reglasRank = [[0 , 0.2, 0.6],[0.2 , 0.4, 0.8],[0.6, 0.8, 1]]

In [59]:
def valRank(predR,predC):    
    return reglasRank[predR][predC]

In [60]:
#for predVar in modelos:    
for predVar in cols_y:
    tablaResultados[predVar+'v1']=tablaResultados[predVar+'_pR_Sem'].apply(semaforoVal2Num)
    tablaResultados[predVar+'v2']=tablaResultados[predVar+'_sem2_pC'].apply(semaforoVal2Num)    
    tablaResultados[predVar+'_Rank'] = tablaResultados.apply(lambda x: valRank(x[predVar+'v1'],x[predVar+'v2']), axis=1)  

In [61]:
## ordernar columnas
columnasResultados=[]
for predVar in cols_y:
    auxCol=[predVar+'_pR',predVar+'_pR_Sem',predVar+'_sem2_pC_probR',predVar+'_sem2_pC_probA',predVar+'_sem2_pC',predVar+'_Rank']
    columnasResultados=columnasResultados+auxCol

In [62]:
tablaResultados[columnasResultados].head(3)

mesAnt,OCUPACION_DW-7_pR,OCUPACION_DW-7_pR_Sem,OCUPACION_DW-7_sem2_pC_probR,OCUPACION_DW-7_sem2_pC_probA,OCUPACION_DW-7_sem2_pC,OCUPACION_DW-7_Rank,OCUPACION_DW-8_pR,OCUPACION_DW-8_pR_Sem,OCUPACION_DW-8_sem2_pC_probR,OCUPACION_DW-8_sem2_pC_probA,OCUPACION_DW-8_sem2_pC,OCUPACION_DW-8_Rank,OCUPACION_DW-9_pR,OCUPACION_DW-9_pR_Sem,OCUPACION_DW-9_sem2_pC_probR,OCUPACION_DW-9_sem2_pC_probA,OCUPACION_DW-9_sem2_pC,OCUPACION_DW-9_Rank,OCUPACION_DW-10_pR,OCUPACION_DW-10_pR_Sem,OCUPACION_DW-10_sem2_pC_probR,OCUPACION_DW-10_sem2_pC_probA,OCUPACION_DW-10_sem2_pC,OCUPACION_DW-10_Rank,OCUPACION_DW-11_pR,OCUPACION_DW-11_pR_Sem,OCUPACION_DW-11_sem2_pC_probR,OCUPACION_DW-11_sem2_pC_probA,OCUPACION_DW-11_sem2_pC,OCUPACION_DW-11_Rank,OCUPACION_DW-12_pR,OCUPACION_DW-12_pR_Sem,OCUPACION_DW-12_sem2_pC_probR,OCUPACION_DW-12_sem2_pC_probA,OCUPACION_DW-12_sem2_pC,OCUPACION_DW-12_Rank
COD_NODO,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
01J,0.326445,V,0.001015,0.001672,V,0.0,0.332595,V,0.009629,0.017980,V,0.0,0.342219,V,0.020471,0.016030,V,0.0,0.317749,V,0.019062,0.037177,V,0.0,0.333805,V,0.008415,0.026106,V,0.0,0.340452,V,0.008581,0.046267,V,0.0
01S,0.452395,V,0.001112,0.001561,V,0.0,0.433711,V,0.019364,0.016871,V,0.0,0.428239,V,0.010402,0.017130,V,0.0,0.421707,V,0.011088,0.042812,V,0.0,0.451648,V,0.018257,0.022082,V,0.0,0.468728,V,0.011535,0.024536,V,0.0
02J,0.325362,V,0.001185,0.002022,V,0.0,0.334995,V,0.018874,0.105148,V,0.0,0.342638,V,0.019484,0.032147,V,0.0,0.332294,V,0.025898,0.093686,V,0.0,0.339170,V,0.034324,0.103864,V,0.0,0.337937,V,0.024198,0.280474,V,0.0


In [63]:
nuevasColumnas=[]
for col in columnasResultados:
    n_col=col
    i=0
    for predVar in cols_y:
        n_col=n_col.replace(predVar,date_list[i])
        i=i+1
    nuevasColumnas.append(n_col)

In [64]:
tablaResultados=tablaResultados[columnasResultados]
tablaResultados.columns=nuevasColumnas

In [65]:
tablaResultados.head(3)

,Jan-22_pR,Jan-22_pR_Sem,Jan-22_sem2_pC_probR,Jan-22_sem2_pC_probA,Jan-22_sem2_pC,Jan-22_Rank,Feb-22_pR,Feb-22_pR_Sem,Feb-22_sem2_pC_probR,Feb-22_sem2_pC_probA,Feb-22_sem2_pC,Feb-22_Rank,Mar-22_pR,Mar-22_pR_Sem,Mar-22_sem2_pC_probR,Mar-22_sem2_pC_probA,Mar-22_sem2_pC,Mar-22_Rank,Apr-22_pR,Apr-22_pR_Sem,Apr-22_sem2_pC_probR,Apr-22_sem2_pC_probA,Apr-22_sem2_pC,Apr-22_Rank,May-22_pR,May-22_pR_Sem,May-22_sem2_pC_probR,May-22_sem2_pC_probA,May-22_sem2_pC,May-22_Rank,Jun-22_pR,Jun-22_pR_Sem,Jun-22_sem2_pC_probR,Jun-22_sem2_pC_probA,Jun-22_sem2_pC,Jun-22_Rank
COD_NODO,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
01J,0.326445,V,0.001015,0.001672,V,0.0,0.332595,V,0.009629,0.017980,V,0.0,0.342219,V,0.020471,0.016030,V,0.0,0.317749,V,0.019062,0.037177,V,0.0,0.333805,V,0.008415,0.026106,V,0.0,0.340452,V,0.008581,0.046267,V,0.0
01S,0.452395,V,0.001112,0.001561,V,0.0,0.433711,V,0.019364,0.016871,V,0.0,0.428239,V,0.010402,0.017130,V,0.0,0.421707,V,0.011088,0.042812,V,0.0,0.451648,V,0.018257,0.022082,V,0.0,0.468728,V,0.011535,0.024536,V,0.0
02J,0.325362,V,0.001185,0.002022,V,0.0,0.334995,V,0.018874,0.105148,V,0.0,0.342638,V,0.019484,0.032147,V,0.0,0.332294,V,0.025898,0.093686,V,0.0,0.339170,V,0.034324,0.103864,V,0.0,0.337937,V,0.024198,0.280474,V,0.0


In [66]:
tablaResultados.to_csv('Resultados/resultadosVp.csv', index=True,sep=';')

In [67]:
end = datetime.now()
print(f'Tiempo: {end - start} segundos')

Tiempo: 0:00:38.435540 segundos
